In [1]:
import os
import base64
##https://github.com/awslabs/amazon-bedrock-agent-samples/tree/main/examples/agent_observability/deploy-langfuse-on-ecs-fargate-with-typescript-cdk

secret_key="sk-lf-46839148-58c4-4cef-9115-75623c78553a"
public_key="pk-lf-0a30d748-a9b9-4325-9fd6-fa09a44ff3b5"
os.environ["LANGFUSE_HOST"] = "https://d3og9zzfs5n481.cloudfront.net" 

# Set up endpoint
otel_endpoint = str(os.environ.get("LANGFUSE_HOST")) + "/api/public/otel/v1/traces"

# Create authentication token:

auth_token = base64.b64encode(f"{public_key}:{secret_key}".encode()).decode()
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = otel_endpoint
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {auth_token}"

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from mcp import stdio_client, StdioServerParameters
from strands import Agent
from strands.tools.mcp import MCPClient
from strands_tools import workflow
from mcp.client.streamable_http import streamablehttp_client
import uuid
from strands.models import BedrockModel
from botocore.config import Config

system_prompt = """You are an expert research assistant with deep analytical skills. When presented with a task, follow this structured approach:

you have capability:
<SYSTEM_CAPABILITY>
* You are utilising an Ubuntu virtual machine using Linux architecture with internet access.
* You can feel free to install Ubuntu applications with your bash tool. Use curl instead of wget.
* When viewing a page it can be helpful to zoom out so that you can see everything on the page.  Either that, or make sure you scroll down to see everything before deciding something isn't available.
* When using your computer function calls, they take a while to run and send back to you.  Where possible/feasible, try to chain multiple of these calls all into one function calls request.
* When using firefox, double click to launch the firefox browser
</SYSTEM_CAPABILITY>

<IMPORTANT>
* Don't assume an application's coordinates are on the screen unless you saw the screenshot. To open an application, please take screenshot first and then find out the coordinates of the application icon. 
* When using Firefox, if a startup wizard or Firefox Privacy Notice appears, IGNORE IT.  Do not even click "skip this step".  Instead, click on the address bar where it says "Search or enter address", and enter the appropriate search term or URL there. Maximize the Firefox browser window to get wider vision.
* If the item you are looking at is a pdf, if after taking a single screenshot of the pdf it seems that you want to read the entire document instead of trying to continue to read the pdf from your screenshots + navigation, determine the URL, use curl to download the pdf, install and use pdftotext to convert it to a text file, and then read that text file directly with your StrReplaceEditTool.
* After each step, take a screenshot and carefully evaluate if you have achieved the right outcome. Explicitly show your thinking: "I have evaluated step X..." If not correct, try again. Only when you confirm a step was executed correctly should you move on to the next one.
* If you encounter CAPTCHA in the website, please stop and hande over to user.
</IMPORTANT>
"""

bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    cache_tools="default",
    boto_client_config=Config(
            retries={
                "max_attempts": 3,
                "mode": "standard",
            },
            read_timeout=600,
            connect_timeout=30,
        ),
)
session_id = str(uuid.uuid4())
# Connect to an MCP server using stdio transport
stdio_mcp_client_1 = MCPClient(lambda: stdio_client(
    StdioServerParameters(command="uv", 
                          args=[
                                "--directory",
                                "/fsx/mcp_projects/sample-mcp-servers/remote_computer_use",
                                "run",
                                "server_claude.py"
                                ],
                          env={
                              "VNC_HOST":"127.0.0.1",
                            "VNC_PORT":"5901",
                            "VNC_USERNAME":"vnc_user",
                            "VNC_PASSWORD":"12345670",
                            "PEM_FILE":"",
                            "SSH_PORT":"2222",
                            "DISPLAY_NUM":"1",
                            "WIDTH":"1024",
                            "HEIGHT":"768"
                          })
))

stdio_mcp_client_2 = MCPClient(lambda: stdio_client(
    StdioServerParameters(command="uv", 
                          args=["--directory","/fsx/mcp_projects/sample-mcp-servers/html_render_service/src",
			"run",
			"server.py"],
                          env={
                          })
))
# streamable_http_mcp_client = MCPClient(lambda: streamablehttp_client(
#     url="https://har89we10a.execute-api.us-east-1.amazonaws.com/Prod/mcp",
#     headers={"Authorization":"Bearer 123456"}))


# Create an agent with MCP tools
with stdio_mcp_client_1:
    with stdio_mcp_client_2:
        # Get the tools from the MCP server
        tools = stdio_mcp_client_1.list_tools_sync()+stdio_mcp_client_2.list_tools_sync()
        for tool in tools:
            print(tool.tool_name, tool.tool_spec)
            
        researcher = Agent(
                model = bedrock_model,
                system_prompt=system_prompt,
                tools=tools, 
                trace_attributes={
                "session.id": session_id,
                "user.id": "river@domain.com",
                "tags": [
                    "Agent-SDK",
                    "Okatank-Project",
                    "Observability-Tags",
        ]
        })
        
        # researcher(f"帮我总结下今天Amazon最新的deals里关于耳机的商品，列出一个top 3清单")
        researcher("帮我制作一份司美格鲁肽的介绍，包括特色功能，适用范围，发展历史，价格，图文并茂，需要制作成精美的 HTML,并生成链接。 请使用bing搜索去查询资料，")

computer {'inputSchema': {'json': {'properties': {'action': {'anyOf': [{'enum': ['key', 'type', 'mouse_move', 'left_click', 'left_click_drag', 'right_click', 'middle_click', 'double_click', 'screenshot', 'cursor_position'], 'type': 'string'}, {'enum': ['left_mouse_down', 'left_mouse_up', 'scroll', 'hold_key', 'wait', 'triple_click'], 'type': 'string'}], 'title': 'Action'}, 'coordinate': {'default': None, 'items': {'type': 'integer'}, 'title': 'Coordinate', 'type': 'array'}, 'duration': {'anyOf': [{'type': 'integer'}, {'type': 'number'}, {'type': 'null'}], 'default': None, 'title': 'Duration'}, 'scroll_direction': {'anyOf': [{'enum': ['up', 'down', 'left', 'right'], 'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Scroll Direction'}, 'scroll_amount': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'default': None, 'title': 'Scroll Amount'}, 'text': {'default': None, 'title': 'Text', 'type': 'string'}}, 'required': ['action'], 'title': 'computerArguments', 'type': 'obj